In [1]:
import pandas as pd

## Data Cleaning 

In [2]:
# loading data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table = pd.read_html(url, header=0,keep_default_na=False) 
df = table[0]

In [3]:
#chaning column names
df.columns = ['PostalCode','Borough','Neighborhood']
#removing irrelavent data
df=df[df.Borough != 'Not assigned']

In [4]:
#formatting data
df = df.groupby(['PostalCode'], sort=False).agg({'Borough':'first','Neighborhood':','.join})
df.Neighborhood = df.Borough.where(df.Neighborhood == 'Not assigned', df.Neighborhood)
df.head()

,Neighborhood,Borough
PostalCode,,
M3A,Parkwoods,North York
M4A,Victoria Village,North York
M5A,"Harbourfront,Regent Park",Downtown Toronto
M6A,"Lawrence Heights,Lawrence Manor",North York
M7A,Queen's Park,Queen's Park


In [5]:
#shape of data
df.shape

(103, 2)

## Geodata

In [6]:
df_g=pd.read_csv('https://cocl.us/Geospatial_data') 
df_g.columns = ['PostalCode','Latitude','Longitude']

In [7]:
df_g.set_index(['PostalCode'], inplace=True)

df_g.head()

In [8]:
#merging geodata with data
df_f = pd.merge(df,df_g,left_index=True,right_index=True)

In [9]:
df_f.head()

,Neighborhood,Borough,Latitude,Longitude
PostalCode,,,,
M3A,Parkwoods,North York,43.753259,-79.329656
M4A,Victoria Village,North York,43.725882,-79.315572
M5A,"Harbourfront,Regent Park",Downtown Toronto,43.654260,-79.360636
M6A,"Lawrence Heights,Lawrence Manor",North York,43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Creating borough map

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[43.65, 79.38]], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_f['Latitude'], df_f['Longitude'], df_f['Borough'], df_f['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [10]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

## forusquare api


In [12]:
CLIENT_ID = 'QG3FAGY2LNM1N3OZD4N31UUARD2CV3NAOB0C2M0L0YZ4XBYT' # your Foursquare ID
CLIENT_SECRET = 'SZJDXIK31NZCBL3WIXHHF35Z4VBAYXL3DAIROEYX33O2OGCO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=df_f['Neighborhood'],
                                   latitudes=df_f['Latitude'],
                                   longitudes=df_f['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [28]:
toronto_venues.groupby('Neighborhood').count()
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()


In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0        Steakhouse  0.10
1              Café  0.07
2       Pizza Place  0.07
3             Hotel  0.07
4  Asian Restaurant  0.07


----Agincourt----
            venue  freq
0          Lounge  0.25
1  Breakfast Spot  0.25
2    Skating Rink  0.25
3  Sandwich Place  0.25
4     Yoga Studio  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
            venue  freq
0     Coffee Shop  0.33
1      Playground  0.33
2            Park  0.33
3   Movie Theater  0.00
4  Massage Studio  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.22
1            Beer Store  0.11
2   Fried Chicken Joint  0.11
3  Fast Food Restaurant  0.11
4              Pharmacy  0.11


----Alderwood,Long Branch----
                venue  freq
0         Pizza Place   0.2
1         Coffee Shop   0.1
2  Athletics & Sports   0.1
3   

                        venue  freq
0              Baseball Field   0.5
1  Construction & Landscaping   0.5
2                 Yoga Studio   0.0
3               Movie Theater   0.0
4              Massage Studio   0.0


----Fairview,Henry Farm,Oriole----
            venue  freq
0  Clothing Store  0.17
1     Coffee Shop  0.13
2   Movie Theater  0.03
3        Tea Room  0.03
4   Smoothie Shop  0.03


----First Canadian Place,Underground city----
           venue  freq
0           Café  0.13
1    Coffee Shop  0.10
2  Deli / Bodega  0.10
3     Steakhouse  0.07
4            Pub  0.03


----Flemingdon Park,Don Mills South----
              venue  freq
0        Beer Store  0.10
1               Gym  0.10
2       Coffee Shop  0.10
3  Asian Restaurant  0.10
4     Grocery Store  0.05


----Forest Hill North,Forest Hill West----
              venue  freq
0  Sushi Restaurant  0.25
1             Trail  0.25
2     Jewelry Store  0.25
3          Bus Line  0.25
4       Yoga Studio  0.00


----Glencairn---

               venue  freq
0  Health Food Store  0.25
1                Pub  0.25
2        Music Venue  0.25
3        Yoga Studio  0.00
4      Movie Theater  0.00


----The Beaches West,India Bazaar----
                venue  freq
0                Park  0.09
1      Sandwich Place  0.09
2       Movie Theater  0.05
3  Light Rail Station  0.05
4          Steakhouse  0.05


----The Danforth West,Riverdale----
                venue  freq
0    Greek Restaurant  0.27
1      Ice Cream Shop  0.07
2  Italian Restaurant  0.07
3     Bubble Tea Shop  0.03
4         Coffee Shop  0.03


----The Junction North,Runnymede----
                  venue  freq
0         Grocery Store  0.25
1     Convenience Store  0.25
2  Caribbean Restaurant  0.25
3              Bus Line  0.25
4         Moving Target  0.00


----The Kingsway,Montgomery Road,Old Mill North----
            venue  freq
0            Park   0.5
1           River   0.5
2     Yoga Studio   0.0
3   Movie Theater   0.0
4  Massage Studio   0.0


----T

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
import numpy as np

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Café,Hotel,Asian Restaurant,Pizza Place,Food Court,Seafood Restaurant,Breakfast Spot,Coffee Shop,Lounge
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Park,Playground,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store,Diner
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Pharmacy,Airport Service,Deli / Bodega
4,"Alderwood,Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Athletics & Sports,Skating Rink,Gym,Pharmacy,Pub,Pool,Women's Store


## Clustering

In [46]:
from sklearn.cluster import KMeans 
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 2, 4, 1, 1, 2, 2, 2, 2, 2], dtype=int32)

In [45]:
toronto_merged = df_f
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
PostalCode,,,,,,,,,,,,,,,
M3A,Parkwoods,North York,43.753259,-79.329656,3.0,Park,Pool,Fast Food Restaurant,Food & Drink Shop,Women's Store,Curling Ice,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
M4A,Victoria Village,North York,43.725882,-79.315572,1.0,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Curling Ice,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Dog Run,Discount Store
M5A,"Harbourfront,Regent Park",Downtown Toronto,43.654260,-79.360636,2.0,Coffee Shop,Park,Bakery,Breakfast Spot,Mexican Restaurant,Café,Yoga Studio,French Restaurant,Farmers Market,Pub
M6A,"Lawrence Heights,Lawrence Manor",North York,43.718518,-79.464763,2.0,Clothing Store,Furniture / Home Store,Arts & Crafts Store,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Accessories Store
M7A,Queen's Park,Queen's Park,43.662301,-79.389494,2.0,Coffee Shop,Gym,Japanese Restaurant,Diner,Yoga Studio,Nightclub,Seafood Restaurant,Burger Joint,Burrito Place,Portuguese Restaurant


In [63]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.65, -79.38], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[cluster-1],
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float